In [93]:
import cvxpy as cp
import numpy as np
from scipy.optimize import minimize

def get_eig(prev_EVs, samples):
    def objective(EV):
        Z = 0
        for vec in samples:
            Z += (EV@vec)**2
        return -Z
    def unit_norm(EV):
        return np.linalg.norm(EV) - 1
    constraints = [{'type': 'eq', 'fun': unit_norm}]
    for vec in prev_EVs:
        constraints.append({
            'type': 'eq',
            'fun': lambda EV, vec=vec: np.dot(EV, vec)
        })
    init = np.random.randn(len(samples[0]))
    init /= np.linalg.norm(init)

    res = minimize(objective, init, constraints=constraints)

    if not res.success:
        raise ValueError("Optimization failed:", res.message)

    EV = res.x / np.linalg.norm(res.x) 
    Eval = -res.fun/(len(samples)-1)
    return Eval, EV

def get_all_eig(samples):
    n_features = len(samples[0])
    eig_vals = []
    eig_vecs = []
    prev_EVs = []

    for i in range(n_features):
        Eval, EV = get_eig(prev_EVs, samples)
        eig_vals.append(Eval)
        eig_vecs.append(EV)
        prev_EVs.append(EV.copy())

    return np.array(eig_vals), np.array(eig_vecs)



In [95]:
L = [
    np.array([1, 2, 3]),
    np.array([0, 1, 0]),
    np.array([2, 0, 1])
]
L_centered = L - np.mean(L, axis=0)
Evals, EVs = get_all_eig(L_centered)
print(Evals)
print(EVs)



[2.89314982e+00 1.44018351e+00 1.73771409e-09]
[[-0.11929055 -0.43891507 -0.89057472]
 [-0.81592772  0.55442032 -0.16395139]
 [-0.56571346 -0.70708675  0.42426008]]


In [33]:
import numpy as np

# Step 1: Data
L = np.array([
    [1, 2, 3],
    [0, 1, 0],
    [2, 0, 1]
])

# Step 2: Center the data
L_centered = L - np.mean(L, axis=0)

# Step 3: Covariance matrix
cov = np.cov(L_centered, rowvar=False)

# Step 4: Eigen decomposition
eigenvalues, eigenvectors = np.linalg.eig(cov)

# Step 5: Get the index of the largest eigenvalue
max_idx = np.argmax(eigenvalues)

# Step 6: Print
print(eigenvalues)
print(eigenvectors)


[0.         1.44018351 2.89314982]
[[-0.56568542 -0.81594751  0.11928813]
 [-0.70710678  0.55439511  0.43891465]
 [ 0.42426407 -0.16393817  0.89057525]]


In [36]:
def max_SS(samples):
    v_sum = np.sum(samples, axis=0)
    unit_EV = v_sum/np.linalg.norm(v_sum)
    return unit_EV

In [ ]:
# Import packages.
import cvxpy as cp
import numpy as np

sum_vec = np.sum(samples, axis=0)
EV = cp.Variable(len(sum_vec))
Z = cp.Maximize(EV @ sum_vec)

const_1 = cp.norm(EV, 2) == 1
orth_const = []
for vec in prev_EVs:
    orth_const.append(EV@vec==0)
constraints = [
    const_1,
    *orth_const
]

L = [
    np.array([1, 2, 3]),
    np.array([0, 1, 0]),
    np.array([2, 0, 1])
]
L_centered = L - np.mean(L, axis=0)
print(L_centered)
Evals, EVs = get_all_eig(L_centered)
print(Evals)
print(EVs)



In [ ]:


# Generate data.
m = 20
n = 15
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Define and solve the CVXPY problem.
x = cp.Variable(n)
cost = cp.sum_squares(A @ x - b)
prob = cp.Problem(cp.Minimize(cost))
prob.solve()

# Print result.
print("\nThe optimal value is", prob.value)
print("The optimal x is")
print(x.value)
print("The norm of the residual is ", cp.norm(A @ x - b, p=2).value)


The optimal value is 7.005909828287485
The optimal x is
[ 0.17492418 -0.38102551  0.34732251  0.0173098  -0.0845784  -0.08134019
  0.293119    0.27019762  0.17493179 -0.23953449  0.64097935 -0.41633637
  0.12799688  0.1063942  -0.32158411]
The norm of the residual is  2.6468679280023557
